In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
import pandas as pd

api_key = 'dMlhEPf9ygVdsnK1fGe4v8cAAmxTKesksyQkHuLk'
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

parameters = {
    'api_key': api_key,
    'fuel_type': 'ELEC',
    'state': 'CA',
}

response = requests.get(url, params=parameters)

if response.status_code == 200:
    # convert JSON data to DataFrame
    data = response.json()
    df_elec = pd.json_normalize(data['fuel_stations'])
    # save DataFrame as CSV file
    #df.to_csv('alt_fuel_stations.csv', index=False)
else:
    print('Error:', response.status_code)


In [ ]:
import requests
import pandas as pd

api_key = 'dMlhEPf9ygVdsnK1fGe4v8cAAmxTKesksyQkHuLk'
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

parameters = {
    'api_key': api_key,
    'state': 'CA',
}

response = requests.get(url, params=parameters)

if response.status_code == 200:
    # convert JSON data to DataFrame
    data = response.json()
    df_all = pd.json_normalize(data['fuel_stations'])
    # save DataFrame as CSV file
    #df.to_csv('alt_fuel_stations.csv', index=False)
else:
    print('Error:', response.status_code)


In [ ]:
df_elec.isna().sum()

access_code                   0
access_days_time           1756
access_detail_code        15521
cards_accepted            15486
date_last_confirmed           3
                          ...  
ev_network_ids.station     2378
ev_network_ids.posts       3079
federal_agency.id         15932
federal_agency.code       15932
federal_agency.name       15932
Length: 74, dtype: int64

In [ ]:
df_elec_filtered=df_elec[['station_name','latitude', 'longitude', 'city', 'zip', 'ev_dc_fast_num',
       'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_network']].copy(deep=True)

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
df_elec_filtered.isna().sum()

station_name              0
latitude                  0
longitude                 0
city                      0
zip                       0
ev_dc_fast_num        14211
ev_level1_evse_num    16056
ev_level2_evse_num     1698
ev_network                0
dtype: int64

In [ ]:
df_elec_filtered['ev_dc_fast_num'] = df_elec_filtered['ev_dc_fast_num'].fillna(df_elec_filtered['ev_dc_fast_num'].median())
df_elec_filtered['ev_level1_evse_num'] = df_elec_filtered['ev_level1_evse_num'].fillna(df_elec_filtered['ev_level1_evse_num'].median())
df_elec_filtered['ev_level2_evse_num'] = df_elec_filtered['ev_level2_evse_num'].fillna(df_elec_filtered['ev_level2_evse_num'].median())


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the data
# Replace this with the actual data loading process
# df_elec_filtered = pd.read_csv("your_data.csv")

# Define the input features and handle missing values
features = ['latitude', 'longitude']
df_elec_filtered = df_elec_filtered.dropna(subset=features)

# Normalize the data
scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df_elec_filtered[features]), columns=features)

# Train the KMeans model
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df_norm)
df_elec_filtered['cluster'] = kmeans.labels_

def recommend_location(latitude, longitude):
    location = pd.DataFrame({'latitude': [latitude], 'longitude': [longitude]})
    location_norm = pd.DataFrame(scaler.transform(location), columns=features)
    location_cluster = kmeans.predict(location_norm)[0]
    cluster_stations = df_elec_filtered[df_elec_filtered['cluster'] == location_cluster]
    cluster_stations['distance'] = ((cluster_stations['latitude'] - latitude)**2 + (cluster_stations['longitude'] - longitude)**2)**0.5
    recommendations = cluster_stations.sort_values('distance').head(10)
    return recommendations[['station_name', 'city', 'zip', 'ev_network', 'distance']]

# Example usage
existing_latitude = 37.7749
existing_longitude = -122.4194
recommendations = recommend_location(existing_latitude, existing_longitude)
print(recommendations)


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                         station_name           city    zip  \
11484             Treasure Island JCC  San Francisco  94130   
5608   City of San Francisco - 49 SVN  San Francisco  94102   
15103     HUDSON PACIFIC 1455 LOBBY 2  San Francisco  94103   
15102     HUDSON PACIFIC 1455 LOBBY 1  San Francisco  94103   
5762    HUDSON PACIFIC 1455 STATION 1  San Francisco  94103   
5766    HUDSON PACIFIC 1455 STATION 2  San Francisco  94103   
5759    HUDSON PACIFIC 1455 STATION 3  San Francisco  94103   
5765    HUDSON PACIFIC 1455 STATION 4  San Francisco  94103   
5760    HUDSON PACIFIC 1455 STATION 5  San Francisco  94103   
5764    HUDSON PACIFIC 1455 STATION 6  San Francisco  94103   

                ev_network  distance  
11484        Non-Networked  0.000034  
5608             POWERFLEX  0.001141  
15103  ChargePoint Network  0.001457  
15102  ChargePoint Network  0.001475  
5762   ChargePoint Network  0.002140  
5766   ChargePoint Network  0.002153  
5759   ChargePoint Network  0.002

<ipython-input-12-10e04c2968f8>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_stations['distance'] = ((cluster_stations['latitude'] - latitude)**2 + (cluster_stations['longitude'] - longitude)**2)**0.5
